In [ ]:
# !pip install --upgrade openai
import os
import re
import glob
import time
import asyncio
import nest_asyncio
import numpy as np
import pandas as pd
from openai import OpenAI, AsyncOpenAI
nest_asyncio.apply()

In [ ]:
path = 'transcripts/'
files = glob.glob(path+'AAPL*')
# files = os.listdir(fp)
files

In [ ]:
async def oai_request(symbol, date, title, prompt):
    client = AsyncOpenAI()
    completion = await client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "You are an analytical assistant."},
        {"role": "user", "content": prompt}
      ]
    )
    await asyncio.sleep(0.5)
    return [symbol, date, title, completion.choices[0].message.content]

In [ ]:
async def requests(fp):
    with open(fp, 'r') as f:
        transcript = f.read().replace('\n', ' ')
    _, fn = fp.split(path)
    symbol, date, title = fn.split('_', maxsplit=2)
    title = title[:-4]
    # split transcripts into 10 chunks and summarize
    # because it exceeds max input tokens
    words = transcript.split(' ')
    chunks = np.array_split(words, 10)
    base_prompt = ('Please only respond with a number. '
                   'What is the number of AI keywords or '
                   'products in this text?: ')
    reqs = []
    for c in chunks:
        prompt = base_prompt+' '.join(c)
        reqs.append(await oai_request(symbol, date, title, prompt))
    return reqs

In [ ]:
%%time
async def first_half():
    tasks = [requests(fp) for fp in files[:6]]
    responses = await asyncio.gather(*tasks)
    return responses

async def second_half():
    tasks = [requests(fp) for fp in files[6:]]
    responses = await asyncio.gather(*tasks)
    return responses

l1 = asyncio.run(first_half())
time.sleep(60)
l2 = asyncio.run(second_half())
l = l1+l2

In [ ]:
posts = [y for x in l for y in x]

In [ ]:
df = pd.DataFrame(posts, columns=['symbol', 'date', 'title', 'ai_count'])
df['date'] = pd.to_datetime(df['date'])
df['ai_count'] = df['ai_count'].apply(lambda x: re.findall(r'\d+', x))
df['ai_count'] = df['ai_count'].str[0].fillna(0).astype(int)

In [ ]:
df.tail()

In [ ]:
grp = df.groupby(['symbol', 'date'])['ai_count'].sum()
grp

In [ ]:
grp.loc['AAPL'].plot()